In [2]:
import io
import re
import string
import tqdm
import os
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd
import numpy as np
import math

In [3]:
# criando dataframe (NOME DO ARQUIVO, SIMILARIDADE, TEXTO)
documentos_df = pd.DataFrame(columns=['Nome', 'Similaridade', 'Texto', 'Tokens'])

In [4]:
# tokeniza o texto 
def tokenize(text):
    pattern = re.compile(r'[A-Za-z]+[\w^\']*|[\w^\']*[A-Za-z]+[\w^\']*')
    return pattern.findall(text.lower())

In [5]:
def getFileName(path):
    name = os.path.basename(path)
    return os.path.splitext(name)[0]

In [ ]:
def similarity(modelSave,tokens,tokens_seach):
  similarity_array = []
  for doc in tokens:
    sim = modelSave.similarity_unseen_docs(tokens_seach,doc)
    similarity_array.append(sim)
  return similarity_array

In [6]:
docs_folder = os.path.join(os.getcwd(), 'docs')

files_path = [] # conterá os caminhos absolutos de todos os arquivos dentro da pasta docs
for root, dirs, files in os.walk(docs_folder):
    for file in files:
        if file.endswith(".txt"):
             files_path.append(os.path.join(root, file))

In [7]:
files_names = []
for path in files_path:
    files_names.append(getFileName(path))

documentos_df['Nome'] = files_names
documentos_df

,Nome,Similaridade,Texto,Tokens
0,doc13,NaN,NaN,NaN
1,doc9,NaN,NaN,NaN
2,doc18,NaN,NaN,NaN
3,doc7,NaN,NaN,NaN
4,doc5,NaN,NaN,NaN
5,doc4,NaN,NaN,NaN
6,doc8,NaN,NaN,NaN
7,doc3,NaN,NaN,NaN
8,doc10,NaN,NaN,NaN
9,doc15,NaN,NaN,NaN


In [8]:
doc_texts = []
for file in files_path:
    with open(file, mode='r') as f:
        doc_texts += [f.read()]

documentos_df['Texto'] = doc_texts
# text = ' '.join(text) # pra juntar tudo numa unica string
# print(len(doc_texts))

documentos_df

,Nome,Similaridade,Texto,Tokens
0,doc13,NaN,Houve um tempo em que os discípulos de Jesus a...,NaN
1,doc9,NaN,A espera do Natal? Você sabia que o Natal é o ...,NaN
2,doc18,NaN,"Era outro dia lindo, e Jesus estava tentando e...",NaN
3,doc7,NaN,"Certa vez, Deus estava olhando o mundo e ficou...",NaN
4,doc5,NaN,"Certo dia, um homem perguntou para Jesus o que...",NaN
5,doc4,NaN,Era uma vez uma mulher muito devota que tinha ...,NaN
6,doc8,NaN,"Saul era rei de Israel, mas vivia longe das le...",NaN
7,doc3,NaN,Jonas era um profeta que dedicava sua vida a e...,NaN
8,doc10,NaN,Você conhece os milagres que Jesus realizou? Q...,NaN
9,doc15,NaN,A Bíblia está cheia de histórias sobre mulhere...,NaN


In [11]:
tokens = []
for text in doc_texts:
    tokens.append(tokenize(text))

documentos_df['Tokens'] = tokens
documentos_df

,Nome,Similaridade,Texto,Tokens
0,doc13,NaN,Houve um tempo em que os discípulos de Jesus a...,"[houve, um, tempo, em, que, os, discípulos, de..."
1,doc9,NaN,A espera do Natal? Você sabia que o Natal é o ...,"[a, espera, do, natal, você, sabia, que, o, na..."
2,doc18,NaN,"Era outro dia lindo, e Jesus estava tentando e...","[era, outro, dia, lindo, e, jesus, estava, ten..."
3,doc7,NaN,"Certa vez, Deus estava olhando o mundo e ficou...","[certa, vez, deus, estava, olhando, o, mundo, ..."
4,doc5,NaN,"Certo dia, um homem perguntou para Jesus o que...","[certo, dia, um, homem, perguntou, para, jesus..."
5,doc4,NaN,Era uma vez uma mulher muito devota que tinha ...,"[era, uma, vez, uma, mulher, muito, devota, qu..."
6,doc8,NaN,"Saul era rei de Israel, mas vivia longe das le...","[saul, era, rei, de, israel, mas, vivia, longe..."
7,doc3,NaN,Jonas era um profeta que dedicava sua vida a e...,"[jonas, era, um, profeta, que, dedicava, sua, ..."
8,doc10,NaN,Você conhece os milagres que Jesus realizou? Q...,"[você, conhece, os, milagres, que, jesus, real..."
9,doc15,NaN,A Bíblia está cheia de histórias sobre mulhere...,"[a, bíblia, está, cheia, de, histórias, sobre,..."


In [13]:
tagged_data = [TaggedDocument(words=_d, tags=[str(i)]) for i, _d in enumerate(tokens)]

print(len(tagged_data))

18


In [14]:
max_epochs = 100

In [15]:
model = Doc2Vec(
    tagged_data, 
    vector_size=3000, 
    alpha=0.001,
    window=2, 
    epochs=100,
    min_alpha=0.00025,
    min_count=1, 
)

In [16]:
model.build_vocab(tagged_data)

In [17]:
for epoch in range(max_epochs):
    print('Época {0}'.format(epoch))
    model.train(
        corpus_iterable=tagged_data,
        epochs=model.epochs,
        start_alpha=model.alpha,
        total_examples=170,
        end_alpha=0.0001
    )

model.save('word2vec.model')

Época 0
Época 1
Época 2
Época 3
Época 4
Época 5
Época 6
Época 7
Época 8
Época 9
Época 10
Época 11
Época 12
Época 13
Época 14
Época 15
Época 16
Época 17
Época 18
Época 19
Época 20
Época 21
Época 22
Época 23
Época 24
Época 25
Época 26
Época 27
Época 28
Época 29
Época 30
Época 31
Época 32
Época 33
Época 34
Época 35
Época 36
Época 37
Época 38
Época 39
Época 40
Época 41
Época 42
Época 43
Época 44
Época 45
Época 46
Época 47
Época 48
Época 49
Época 50
Época 51
Época 52
Época 53
Época 54
Época 55
Época 56
Época 57
Época 58
Época 59
Época 60
Época 61
Época 62
Época 63
Época 64
Época 65
Época 66
Época 67
Época 68
Época 69
Época 70
Época 71
Época 72
Época 73
Época 74
Época 75
Época 76
Época 77
Época 78
Época 79
Época 80
Época 81
Época 82
Época 83
Época 84
Época 85
Época 86
Época 87
Época 88
Época 89
Época 90
Época 91
Época 92
Época 93
Época 94
Época 95
Época 96
Época 97
Época 98
Época 99


In [19]:
modelSave = Doc2Vec.load('word2vec.model')

In [18]:
query = 'Jesus Cristo'

tokens_seach = tokenize(query)
tokens_seach

['jesus', 'cristo']